# Uso de MyFitnessPal para conseguir la información nutricional de mi comida

Vamos a utilizar la librería de Python que tiene OpenAI que nos facilita la vida para utilizar uno de los modelos de ChatGPT.

In [141]:
import requests
import os
import openai
import json
from openai import OpenAI

Para esta demo, necesito dos api keys, una es la de Open AI, que me permite hacer peticiones a la API.
La otra es la de Rapid API que viene muy bien para hacer demos o pruebas, tenéis a disposición múltiples apis y muchas son gratuitas o tienen un tier gratuito. Esto nos permite probar cosas sin tener que pagar o registrarnos en todas.
El siguiente código carga mi fichero con las variables de entorno `.env` donde tengo metidas las dos api keys que necesito.

In [142]:
from dotenv import load_dotenv
load_dotenv()

True

Cargamos las variables de entorno para poder utilizarlas más cómodamente. Creamos el `client` de la librería de OpenAI. Por defecto, utilizará OPENAI_API_KEY cuando necesito la api key para conectarse. Si lo tenéis metido en otra variable, habrá que pasárselo como parámetro.

In [143]:
# Set your OpenAI and MyFitnessPal API keys
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
RAPIDAPI_KEY = os.environ.get('RAPIDAPI_KEY')

client = OpenAI()
model = "gpt-4o-mini"

Vamos a escribir primero una función para contactar con la api de MyFitnessPal y recuperar la información nutricional de un alimento.
Esta función es muy sencilla, hace una petición a la api de MyFitnessPal para buscar por palabra clave. Devuelve un array de jsons con los resultados que ha encontrado. Aquí vamos a utilizar por defecto el primer resultado.

In [144]:
def fetch_nutrition_data(keyword):
    url = "https://myfitnesspal2.p.rapidapi.com/searchByKeyword"
    headers = {
        "x-rapidapi-host": "myfitnesspal2.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY,        
    }
    params = {
        "keyword": keyword,
        "page": 1
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API Error: {response.status_code} - {response.text}"}

Ahora vamos a definir qué tool queremos proporcionar al modelo. En nuestro caso, queremos hacer una llamada a la api de MyFitnessPal, para eso necesitamos la palabra clave `keyword` y la cantidad tomada `quantity`. Con esto, el modelo sabe que puede llamar a esta `tool` cuando un usuario le dice lo que ha tomado.

In [145]:
from pydantic import BaseModel

class FoodItem(BaseModel):
    keyword: str
    quantity: int

tools = [openai.pydantic_function_tool(FoodItem)]

Por último, necesitamos una función para contactar con chat gpt. El primer prompt que le vamos a pasar a chat gpt es con el `role: system`. Este le dará a chat gpt unas instrucciones de lo que esperamos del modelo. Qué queremos que haga? Qué debe hacer con lo que le diga el usuario? En este caso utilizamos un texto muy corto. Veréis que no hace falta escribir textos muy largos para conseguir buenos resultados rápidamente pero si queréis hacer cosas más avanzadas, quizás necesitéis darle alguna vuelta.

In [146]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant that helps summarize nutritional information. Use the supplied tools to get the nutrition data of the food items."
    }
]

def parse_food_items(user_input):
    messages.append({
        "role": "user",
        "content": user_input
    })
        
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    
    return chat_completion

Ahora viene lo que es la interacción con el modelo. Primero pediremos al usuario que nos diga qué ha comido. Podríamos haber instruido al modelo para que lo haga pero es más sencillo pedirlo directamente.

In [148]:
user_input = input("What are you eating?").strip()

What are you eating? I'm eating two oreos.


In [149]:
chat_completion = parse_food_items(user_input)

In [150]:
print(chat_completion)

ChatCompletion(id='chatcmpl-AcsR5J7Jow4Gu9BWZhZ2Jb14b2ah5', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2zHaPNGIg8USrTCR01IbznLE', function=Function(arguments='{"keyword": "oreo", "quantity": 2}', name='FoodItem'), type='function')]))], created=1733829079, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_bba3c8e70b', usage=CompletionUsage(completion_tokens=35, prompt_tokens=80, total_tokens=115, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


Podemos ver que el modelo ha detectado que tiene que llamar a una `tool` y ha formateado el json correctamente con el keyword y la cantidad.

El siguiente código comprueba que se ha llamado a la `tool` y si es así, busca toda la información necesaria contactando con la api. Después genera un objeto con el resultado obtenido, cogemos el primero del array que devuelve la api de MyFitnessPal y lo guardamos.

In [151]:
if not chat_completion.choices[0].message.tool_calls:
    print(chat_completion.choices[0].message.content)
else:
    function_call_result_messages = []
    for tool_call in chat_completion.choices[0].message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        print(f"Searching nutrition data for {arguments['keyword']} (Quantity: {arguments['quantity']})...")
        nutrition_data = fetch_nutrition_data(arguments['keyword'])
        
        function_call_result_messages.append({
            "role": "assistant",
            "content": json.dumps(nutrition_data[0])
        })

print(function_call_result_messages)

Searching nutrition data for oreo (Quantity: 2)...
[{'role': 'assistant', 'content': '{"name": "Oreos", "brand": "Generic", "nutrition": {"Serving Size": "3 cookies", "Calories": "157.8", "Fat": "6.5g", "Carbs": "24.1g", "Protein": "1.8g"}}'}]


Ahora vamos a recrear el chat con los mensajes que envió el usuario pero el último mensaje añadirá la información nutricional y el modelo podrá responder con estos datos al usuario.

In [152]:
completion_payload = {
    "model": model,
    "messages": messages + function_call_result_messages
}
print(completion_payload)

{'model': 'gpt-4o-mini', 'messages': [{'role': 'system', 'content': 'You are an assistant that helps summarize nutritional information. Use the supplied tools to get the nutrition data of the food items.'}, {'role': 'user', 'content': "I'm eating two oreos."}, {'role': 'assistant', 'content': '{"name": "Oreos", "brand": "Generic", "nutrition": {"Serving Size": "3 cookies", "Calories": "157.8", "Fat": "6.5g", "Carbs": "24.1g", "Protein": "1.8g"}}'}]}


Ahora debemos contactar de nuevo con nuestro modelo, pasándole los mensajes y ver qué respuesta le da al usuario.

In [153]:
response = client.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)

In [154]:
print(response.choices[0].message.content)

Two Oreos contain approximately:

- **Calories**: 105.2 kcal
- **Fat**: 4.3 g
- **Carbohydrates**: 16.1 g
- **Protein**: 1.2 g

(Note: These values are based on the nutritional information for three cookies. The values for two cookies are calculated proportionally.)
